## Geo-parsing and Geo-Visualization of Road Traffic Crash Incident Locations from Print Media for Emergency Response and Planning

### Code for RTC-NER Baseline Model

#### Setup the Environment

In [4]:
!python3 -m pip install tensorflow

In [5]:
!pip install spacy-transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 30.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver

In [6]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [7]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
from seqeval.metrics import classification_report #to evaluate model
from spacy.tokens import DocBin #to load model

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Load the annotated Training and Test datasets

In [ ]:
rtc_train = open('rtc_train1.json')
TRAIN_DATA = json.load(rtc_train)

rtc_test = open('rtc_test1.json')
TEST_DATA = json.load(rtc_test)

#### Convert the .json files into .spacy format

In [ ]:
# Create DocBin Objects
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

for text, annot in tqdm(TEST_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./test_data.spacy") # save the docbin object

100%|██████████| 1/1 [00:00<00:00, 61.55it/s]


### Train the RTC-NER Baseline Model

In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2024-01-23 18:40:45.720586: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 18:40:45.720632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 18:40:45.722202: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 18:40:46.787204: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and

In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2024-01-23 18:41:15.429324: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 18:41:15.429380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 18:41:15.431000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 18:41:16.526217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: [

## Evaluation of the RTC-NER Baseline Model

In [13]:
!python -m spacy evaluate model-best/ "./test_data.spacy"

ℹ Using CPU
/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================

### Toponym Recognition Using the RTC-NER Baseline Model

In [9]:
rtc_ner_baseline = spacy.load("model-best")

In [10]:
doc1 = rtc_ner_baseline(""" No fewer than five people were confirmed dead on Sunday while 12 others sustained varying degrees of injuries in an accident near the Foursquare Camp on the Lagos-Ibadan Expressway.

Mrs Florence Okpe, the Public Education Officer of the Federal Road Safety Corps (FRSC) in Ogun, confirmed the development in an interview with newsmen in Abeokuta.

Okpe disclosed that 21 people were involved in the multiple crash, saying that 12 people were injured while five people died.

She stated that the injured people were taken to Victory Hospital in Ogere, near Abeokuta for medical attention while the dead were deposited at a morgue in Ipara community, near Abeokuta.

""")

In [11]:
spacy.displacy.render(doc1, style="ent", jupyter=True) # display in Jupyter

In [14]:
for ent in doc1.ents:
	print(ent.text, " ->>>> ", ent.label_)

Lagos-Ibadan Expressway.  ->>>>  ROAD
Victory Hospital in Ogere,  ->>>>  HOSPITAL
Ipara community,  ->>>>  LANDMARK


### Toponym Resolution (Geocoding)

In [15]:
#Displaying selected RTC location information
locations = []
location_tag = []

tags = ['ROAD', 'LANDMARK', 'TOWN', 'LGA', 'STATE', 'HOSPITAL']

for tag in tags:
  for ent in doc1.ents:
      if ent.label_ == tag:
        locations.append(ent.text)
        #location_tag.append(tag)

#print(locations)
#print(location_tag)

#function to concatenate all elements of locations list (except the last element) as rtc_site
def conexclast(strlst):
    output = ""
    for elem in strlst:
       strng = str(elem)
       output = output+strng
    return ', '.join(strlst[0:-1])


rtc_site = conexclast(locations[1:])
rtc_road = locations[0]
hospital = locations[-1]


rtc_places = []
rtc_places.append(rtc_road)
rtc_places.append(rtc_site)
rtc_places.append(hospital)

print("Road Traffic Crash Location details:", rtc_places)

Road Traffic Crash Location details: ['Lagos-Ibadan Expressway.', 'Ipara community,', 'Victory Hospital in Ogere,']


In [16]:
#Using Googgle maps geocoding api
import requests
import pandas as pd

API_KEY = 'xxxxxxxx' #insert your Google Geocoding API key inplace of xxxxxxxx

df = pd.DataFrame()

latitude_list = []
longitude_list = []

for loc in rtc_places:
  params = {
    'key': API_KEY,
    'address': loc
  }

  base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
  response = requests.get(base_url, params=params).json()

  geometry = response['results'][0]['geometry']
  lat = geometry['location']['lat']
  lon = geometry['location']['lng']
  latitude_list.append(lat)
  longitude_list.append(lon)

df = pd.DataFrame({'Location': rtc_places})

df['Latitude'] = latitude_list
df['Longitude'] = longitude_list

df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

df

,Location,Latitude,Longitude
0,Lagos-Ibadan Expressway.,6.923588,3.636422
1,"Ipara community,",7.002029,3.666126
2,"Victory Hospital in Ogere,",6.936595,3.634447


### Geo-Visualization

In [17]:
import folium

In [18]:
df

,Location,Latitude,Longitude
0,Lagos-Ibadan Expressway.,6.923588,3.636422
1,"Ipara community,",7.002029,3.666126
2,"Victory Hospital in Ogere,",6.936595,3.634447


In [19]:
road_coord = [df['Latitude'].iloc[0], df['Longitude'].iloc[0]]
print(road_coord)

[6.9235877, 3.6364221]


In [20]:
map = folium.Map(location=road_coord, zoom_start=13)

folium.Marker(road_coord, popup='RTC Road').add_to(map)
map

In [21]:
# Coordinates of the points
points = []
for i in range(df.shape[0]):
    rtc_coords = [df['Latitude'].iloc[i], df['Longitude'].iloc[i]]
    points.append(rtc_coords)

points

[[6.9235877, 3.6364221], [7.0020289, 3.6661265], [6.9365954, 3.6344466]]

In [22]:
for point in points:
    folium.Marker(point, popup='Point').add_to(map)
map